Laden der Packages

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as Rot

Funktion zur Bestimmung der Eulerwinkel zwischen zwei Koordinatensystemen

In [2]:
# Funktion zur Berechnung der Euler-Winkel zwischen zwei Rotationsmatrizen
def calculate_euler_angles(matrix1, matrix2, sequence="xyz", degrees=True):
    """
    Berechnet die Euler-Winkel zwischen zwei Rotationsmatrizen.
    
    :param matrix1: Erste Rotationsmatrix (3x3 NumPy-Array)
    :param matrix2: Zweite Rotationsmatrix (3x3 NumPy-Array)
    :param sequence: Rotationsachsenfolge für Euler-Winkel (z.B. "xyz", "zyx")
    :param degrees: Gibt an, ob die Ergebnisse in Grad (True) oder Radiant (False) ausgegeben werden
    :return: Euler-Winkel zwischen den Matrizen als NumPy-Array
    """
    # Berechne die relative Rotationsmatrix
    relative_matrix = np.dot(matrix1.T, matrix2)  # R_rel = R2 * R1^T
    
    # Wandle die Rotationsmatrix in Euler-Winkel um
    rotation = Rot.from_matrix(relative_matrix)
    euler_angles = rotation.as_euler(sequence, degrees=degrees)
    
    return euler_angles

# Funktion zur Berechnung der Rotationsmatrix aus drei Punkten
def create_coordinate_system_polyworks(p1, p2, p3):
    """
    Erstellt eine orthogonale Rotationsmatrix aus drei Punkten wie in PolyWorks:
    - X: p1 -> p2
    - Y: p1 -> p3
    - Z: X x Y (rechtshändiges Koordinatensystem)
    """

    # Richtung X: p1 -> p2
    x_dir = p2 - p1
    x_axis = x_dir / np.linalg.norm(x_dir) # Normierung der X-Achse

    # Richtung Y: p1 -> p3
    y_dir = p3 - p1
    # Noch nicht normalisieren, erst orthogonalisieren!

    # Z-Achse: Kreuzprodukt von X und Y (rechtshändiges System), Z-Achse wird orthogonal zu den vorläufigen X- und Y-Achsen aufgespannt
    z_axis = np.cross(x_axis, y_dir)
    z_axis = z_axis / np.linalg.norm(z_axis) # Normierung der Z-Achse

    # Orthogonalisierung der Y-Achse: Z x X, da X- und Y-Achse nicht orthogonal sind, wird Y orthogonal zu X und Z aufgespannt
    y_axis = np.cross(z_axis, x_axis)
    y_axis = y_axis / np.linalg.norm(y_axis) # Normierung der Y-Achse

    # Rotationsmatrix: [X Y Z] als Spaltenvektoren
    R = np.column_stack((x_axis, y_axis, z_axis))

    return R

Rohdaten laden und Variablen definieren

In [3]:
filename = '../data/Punkte_Koordinatensysteme_Berechnung.csv'
df = pd.read_csv(filename, delimiter=';')
points = df.to_numpy()
points_coord = np.array(points[:,1:], dtype=float)
points_identifiers = points[:,0]
euler_angles_degrees = []
rows, cols = 30, 5  
rotation_matrix_all = pd.DataFrame(np.empty((rows, cols), dtype=object), columns = ['Gelenk', 'Achse', 'X', 'Y', 'Z'])
counter = 0
angles_between_systems = pd.DataFrame(np.empty((9, 4), dtype=object), columns = ['Gelenk', 'Winkel_x', 'Winkel_y', 'Winkel_z'])
translation_between_systems = pd.DataFrame(np.empty((9, 4), dtype=object), columns = ['Gelenk', 'Translation_x', 'Translation_y', 'Translation_z'])

Berechnung der Rotationsmatrizen zwischen den Gelenkkoordinatensystemen

In [4]:
# for-Loop zur Berechnung der Rotationsmatrizen
for i in range(0, len(points), 3):
    required_points = points_coord[i:i+3, :]
    p1 = required_points[0, :]
    p2 = required_points[1, :]
    p3 = required_points[2, :]

    # Rotationsmatrix aufbauen
    R = create_coordinate_system_polyworks(p1, p2, p3)

    # Rotation um 180° um die x-Achse (für Maschinenkkoordinatensystem)
    R_180_x = np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1]
    ])
    # Rotation um -90° um die z-Achse (für Maschinenkoordinatensystem)
    theta_deg = -90  # Winkel in Grad
    theta_rad = np.radians(theta_deg) # Umwandlung in Radiant
    R_z_neg_90 = np.array([[np.cos(theta_rad), -np.sin(theta_rad), 0],
               [np.sin(theta_rad), np.cos(theta_rad), 0],
               [0, 0, 1]]) # Rotationsmatrix um -90° um die z-Achse
    
    # Berechnung der Rotationsmatrix für die Sonderfälle des Maschinenkoordinatensystems (Maschine und Maschine_neu)
    if i == 0:
        R_total = R @ R_180_x
        rotation_matrix_all.iloc[i,0] = 'Maschine'
    elif i == 27:
        R_total = R @ R_180_x
        R_total = R_total @ R_z_neg_90 
        rotation_matrix_all.iloc[i,0] = 'Maschine_neu'
    else:
        R_total = R
        rotation_matrix_all.iloc[i,0] = 'Gelenk' + str(counter-1)
    
    # Zuordnung der Achsenbezeichnungen und der Rotationsmatrix zum DataFrame
    rotation_matrix_all.iloc[i,1] = 'i'
    rotation_matrix_all.iloc[i+1,1] = 'j'
    rotation_matrix_all.iloc[i+2,1] = 'k'
    rotation_matrix_all.iloc[i,2:5] = R_total[0,:]
    rotation_matrix_all.iloc[i+1,2:5] = R_total[1,:]
    rotation_matrix_all.iloc[i+2,2:5] = R_total[2,:]
    
    # Erhöhung des Zählers
    counter = counter + 1
    gelenk_identifier_1 = str(counter - 3)
    gelenk_identifier_2 = str(counter - 2)

    if i == 3:
        gelenk_identifier_1 = 'Maschine'

    # Berechnung der Euler-Winkel zwischen dem vorherigen und dem aktuellen Koordinatensystemen 
    if i > 0 and i < 27:
        angles_between_systems.iloc[counter-2,0] = 'Gelenk_' + gelenk_identifier_1 + '/' + gelenk_identifier_2
        angles_between_systems.iloc[counter-2,1:4]= calculate_euler_angles(R_previous,R_total,sequence="xyz", degrees=False)
        # Berechnung der Translation zwischen den Koordinatensystemen
        translation_between_systems.iloc[counter-2,0] = 'Gelenk_' + gelenk_identifier_1 + '/' + gelenk_identifier_2
        translation_between_systems.iloc[counter-2,1:4] = p1 - p1_previous
    elif i == 27:
        angles_between_systems.iloc[counter-2,0] = 'Maschine_neu/Gelenk_0'
        angles_between_systems.iloc[counter-2,1:4]= calculate_euler_angles(R_total, rotation_matrix_all.iloc[0:3,2:5],sequence="xyz", degrees=False)
        # Berechnung der Translation zwischen den Koordinatensystemen
        translation_between_systems.iloc[counter-2,0] = 'Maschine_neu/Gelenk_0'
        translation_between_systems.iloc[counter-2,1:4] = p1 - points_coord[3, :]
    # Speichern der aktuellen Rotationsmatrix und des Punktes für die nächste Iteration
    R_previous = R_total
    p1_previous = p1
    Gelenk_previous = counter - 2

# Export des DataFrames mit den Rotationsmatrizen in eine CSV-Datei
rotation_matrix_all.to_csv('../results/Tabellen/Rotation_Matrix_Nullposition.csv', float_format='%.10f', sep=';', index=False)
# Export des DataFrames mit den Translationen in eine CSV-Datei
translation_between_systems.to_csv('../results/Tabellen/Translation_Nullposition.csv', float_format='%.10f', sep=';', index=False)
# Export des DataFrames mit den Euler-Winkeln in eine CSV-Datei
angles_between_systems.to_csv('../results/Tabellen/Euler_Winkel_Nullposition.csv', float_format='%.10f', sep=';', index=False)

Verschiebung des Maschinenkoordinatensystems, so dass die z-Achse des Maschinenkoordinatensystems mit der Drehachse von Gelenk 0, die y-Achse mit der Maschinenlängsachse und die x-Achse mit der Maschinenquerachse übereinstimmt (bessere Interpretation der Koordinaten, bei der Vermessung)

In [5]:
# Ermittlung der relativen Rotationsmatrix zwischen Maschine und Maschine_neu
R_machine_old = rotation_matrix_all.iloc[0:3,2:5].to_numpy()
R_machine_new = rotation_matrix_all.iloc[27:30,2:5].to_numpy()
R_machine_relative = np.dot(R_machine_old.T,R_machine_new) # relative Rotationsmatrix zwischen Maschine und Maschine_neu
print(R_machine_relative)

[[0.9584121025300131 0.16756093881601536 -0.23101855662932147]
 [-0.19246142863793156 0.9771983440101739 -0.08967717073157565]
 [0.21072456005564663 0.1304098472005445 0.9688077371400813]]
